In [3]:
import random
import numpy as np
from emergent.main import AgentModel

In [ ]:
def generateInitialData(model: AgentModel):
    initial_data = {
        "a_alpha": random.randint(1, 4),
        "a_beta": random.randint(1, 4),
        "b_alpha": random.randint(1, 4),
        "b_beta": random.randint(1, 4),
    }
    expectations = {
        "a_expectation": initial_data["a_alpha"]
        / (initial_data["a_alpha"] + initial_data["a_beta"]), #A's expectation is its alpha value divided by the alpha + beta values
        "b_expectation": initial_data["b_alpha"]
        / (initial_data["b_alpha"] + initial_data["b_beta"]), #B's expectation is its alpha value divided by the alpha + beta values
    }
    initial_data.update(expectations)
    return initial_data

In [5]:
def generateTimestepData(model: AgentModel):
    graph = model.get_graph()

    for _node, node_data in graph.nodes(data=True):
        # agent pulls the "a" bandit arm
        if node_data["a_expectation"] > node_data["b_expectation"]:
            node_data["a_alpha"] += int(
                np.random.binomial(model["num_trials"], model["a_objective"], size=None)
            )
            # TODO
            node_data["a_beta"] += max(0, model["num_trials"] - node_data["a_alpha"])
            node_data["a_expectation"] = node_data["a_alpha"] / (
                node_data["a_alpha"] + node_data["a_beta"]
            )

        # agent pulls the "b" bandit arm
        else:
            node_data["b_alpha"] += int(
                np.random.binomial(model["num_trials"], model["b_objective"], size=None)
            )
            # TODO
            node_data["b_beta"] += max(0, model["num_trials"] - node_data["b_alpha"])
            node_data["b_expectation"] = node_data["b_alpha"] / (
                node_data["b_alpha"] + node_data["b_beta"]
            )

    model.set_graph(graph)

In [8]:
def constructModel() -> AgentModel:
    # Recreating the Zollman Model
    model = AgentModel()
    model.update_parameters({"a_objective": 0.19, "b_objective": 0.71, "num_trials": 30})

    model.set_initial_data_function(generateInitialData)
    model.set_timestep_function(generateTimestepData)

    return model

In [9]:
model = constructModel()
model["num_nodes"] = 20
model.initialize_graph()

timesteps = 20

for _ in range(timesteps):
    model.timestep()

graph = model.get_graph()

for node, node_data in graph.nodes(data=True):
    print(node_data["a_expectation"], node_data["b_expectation"])

0.3333333333333333 0.983451536643026
0.3333333333333333 0.9679633867276888
0.3870967741935484 0.9696969696969697
0.4 0.9819819819819819
0.5 0.9836065573770492
0.7298850574712644 0.2
0.782051282051282 0.3333333333333333
0.21212121212121213 0.9710144927536232
0.21212121212121213 0.9787735849056604
0.22580645161290322 0.9636803874092009
0.2 0.9705882352941176
0.2 0.9796839729119639
0.3225806451612903 0.9832535885167464
0.25 0.9751693002257337
0.3125 0.9716312056737588
0.5 0.9795918367346939
0.16129032258064516 0.9759036144578314
0.6086956521739131 0.2
0.21212121212121213 0.9696969696969697
0.2 0.9862385321100917
